In [1]:
import { load } from "dotenv";
const env = await load();

const process = { env };
// process.env;


In [ ]:
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";
import { TextLoader } from "langchain/document_loaders/fs/text";
import { OpenAIEmbeddings } from "@langchain/openai";

const loader = new TextLoader("documents/data.txt");
const docs = await loader.load();

const splitter = new RecursiveCharacterTextSplitter({
  chunkSize: 100,
  chunkOverlap: 20,
});

const splitDocs = await splitter.splitDocuments(docs);

const embeddings = new OpenAIEmbeddings({
  model: process.env.EMBEDDING_MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

// embeddings
console.log(embeddings.modelName);

await embeddings.embedQuery(splitDocs[0].pageContent);


In [ ]:
import { ChatMessageHistory } from "langchain/memory";
import { HumanMessage, AIMessage } from "@langchain/core/messages";
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";

const chatHistory = new ChatMessageHistory();

// await chatHistory.addUserMessage('你好')
// await chatHistory.addAIChatMessage('有什么可以帮你的？')

await chatHistory.addMessage(new HumanMessage("hi"));
await chatHistory.addMessage(new AIMessage("What can I do for you?"));

const messages = await chatHistory.getMessages();

// console.log(messages);
const chatModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const prompt = ChatPromptTemplate.fromMessages([
  ["system", "Your are a {systemText}"],
  new MessagesPlaceholder("chat_history"),
  ["human", "{humanText}"],
]);

const chain = prompt.pipe(chatModel);

await prompt.invoke({
  systemText: "Assistant is a large language model",
  humanText: "Hello",
  chat_history: messages,
});


In [ ]:
import { ChatMessageHistory } from "langchain/memory";
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { RunnableWithMessageHistory } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";

const chatHistory = new ChatMessageHistory();

// console.log(messages);
const chatModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const prompt = ChatPromptTemplate.fromMessages([
  ["system", "Your are a chat bot"],
  new MessagesPlaceholder("chat_history"),
  ["human", "{humanText}"],
]);

const outputParse = new StringOutputParser();

const chain = prompt.pipe(chatModel).pipe(outputParse);

const chainWithHistory = new RunnableWithMessageHistory({
  runnable: chain,
  getMessageHistory: () => chatHistory,
  inputMessagesKey: "humanText",
  historyMessagesKey: "chat_history",
});

const res1 = await chainWithHistory.invoke(
  {
    humanText: "Hello, I'm Bob",
  },
  {
    configurable: { sessionId: "none" },
  }
);
console.log(res1);

await chainWithHistory.invoke(
  {
    humanText: "Who am I?",
  },
  {
    configurable: { sessionId: "none" },
  }
);


In [ ]:
import { PromptTemplate } from "@langchain/core/prompts";
import { StringOutputParser } from "@langchain/core/output_parsers";
import {
  RunnableSequence,
  RunnablePassthrough,
} from "@langchain/core/runnables";

const prompt = PromptTemplate.fromTemplate("你好, {name}, 我今年{age}岁了");

// await prompt.invoke({ job: '三甲医院的骨科医生', question: '如何预防半月板损伤' })

// const chain = RunnableSequence.from([
//   {
//     name: (input) => input.name + '111'
//   },
//   (input) => {
//     return { name: '李四' + input.name }
//   },
//   prompt,
//   (input) => {
//     return input.value + '222'
//   }
// ]);

const chain = RunnableSequence.from([
  // input直通管道
  new RunnablePassthrough({
    // func，可以在这里根据input做一些额外的逻辑，不影响input的传递
    func: (input) => {
      console.log("input1---", input);
      // 返回什么都不影响input的结构
      return "666";
    },
  }),
  // 合并input和额外的参数
  RunnablePassthrough.assign({
    age: () => {
      return 16;
    },
    name: (input) => {
      console.log("input2---", input);
      return "王五";
    },
  }),
  {
    abc: (input) => {
      console.log("input---", input);
      return { ...input, name: "李四" };
    },
  },
  (input) => {
    console.log("input---", input);
    return input.abc;
  },
  prompt,
  (input) => {
    return { content: input.value };
  },
  new StringOutputParser(),
]);
await chain.invoke({ name: "张三" });


In [2]:
import { PromptTemplate, ChatPromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { StringOutputParser } from "@langchain/core/output_parsers";
import {
  RunnableSequence,
  RunnablePassthrough,
} from "@langchain/core/runnables";
import { ChatMessageHistory, getBufferString } from "langchain/memory";

const summaryPrompt = PromptTemplate.fromTemplate(
  `
请基于已有摘要和新增对话内容，生成一个连贯的渐进式更新摘要。新摘要需整合历史信息与最新对话要点，保持语义完整性和上下文连贯性。

当前摘要：
{summary}

新增对话内容： 
{new_lines}
`
);

const summaryModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const summaryChain = RunnableSequence.from([
  summaryPrompt,
  summaryModel,
  new StringOutputParser(),
]);

// const newSummary = await summaryChain.invoke({
//   summary: "我很喜欢旅行",
//   new_lines: "我昨天晚上去了北京，很开心",
// });

// console.log(newSummary);

// const summary2 = await summaryChain.invoke({
//   summary: newSummary,
//   new_lines: "我计划今年冬天的时候再去一次",
// });

// console.log(summary2);

// await summaryChain.invoke({
//   summary: summary2,
//   new_lines: "我想去看天安门升旗，什么时候去比较好？",
// });

const chatPrompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "你是一个机器人助理，请尽你所能回答用户的问题，这里有一个关于用户的摘要：{summary}",
  ],
  ["human", "{input}"],
]);

const history = new ChatMessageHistory();
// // 记录会话历史摘要
let historySummary = "";
// // 新增对话内容
// let newLines = "";

const chatModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});
const chatChain = RunnableSequence.from([
  {
    input: new RunnablePassthrough({
      func: (input) => {
        // console.log("用户提问加入会话历史", input);
        // newLines = input;
        history.addUserMessage(input);
      },
    }),
  },
  // 合并总结摘要和用户输入，传递给prompt
  RunnablePassthrough.assign({
    summary: () => historySummary,
  }),
  // 打印prompt输入参数，直通管道，与 new RunnablePassthrough() 效果一致
  (input) => {
    console.log("input", input);
    return input;
  },
  chatPrompt,
  chatModel,
  new StringOutputParser(),
  new RunnablePassthrough({
    func: async (input) => {
      // console.log("机器人回答加入会话历史", input);
      history.addAIChatMessage(input);
      const messages = await history.getMessages()
      // console.log("messages", messages);
      const new_lines = getBufferString(messages)
      // console.log("new_lines", new_lines);
      // 更新总结摘要
      const newSummary = await summaryChain.invoke({
        summary: historySummary,
        new_lines: new_lines
      });
      history.clear();
      historySummary = newSummary;
    },
  }),
]);

// await chatChain.invoke("我想出去玩, 你推荐去哪里玩？");


In [3]:
const rst1 = await chatChain.invoke("我昨天晚上去了北京，很开心");
console.log(rst1 + '\n');
const rst2 = await chatChain.invoke("去爬了长城");
console.log(rst2 + '\n');
const rst3 = await chatChain.invoke("我计划今年冬天的时候再去一次");
console.log(rst3 + '\n');
const rst4 = await chatChain.invoke("我想去看升旗");
console.log(rst4 + '\n');
const rst5 = await chatChain.invoke("会不会很冷");
console.log(rst5 + '\n');
const rst6 = await chatChain.invoke("人会不会很多");
console.log(rst6 + '\n');
const rst7 = await chatChain.invoke("从深圳过去机票会不会很贵");
console.log(rst7 + '\n');







input { input: "我昨天晚上去了北京，很开心", summary: "" }
哇，那太棒啦！北京有好多好玩的地方呢，比如故宫、长城、颐和园等等，你昨天晚上都去了哪些地方呀，有没有什么特别有意思的经历可以和我说说哦。

input {
  input: "去爬了长城",
  summary: "当前摘要：\n" +
    "新增对话内容：人类表示昨天晚上去了北京，感觉很开心。AI回应称很棒，并列举了北京诸多好玩的地方如故宫、长城、颐和园等，还询问人类昨天晚上具体去了哪些地方以及是否有特别有意思的经历。\n" +
    "更新后摘要：人类告知昨天晚上去了北京且玩得很开心，AI对此给予热情回应，提及北京的一些好玩之处，还进一步询问人类昨晚所去的具体地点及有无特别有意思的经历。"
}
哇，爬长城很不错呀！长城雄伟壮观，在上面可以领略到很不一样的风景呢。那爬长城的过程中有没有什么特别有趣的事儿呀，比如遇到了很有意思的游客，或者看到了特别美的景色之类的呢？

input {
  input: "我计划今年冬天的时候再去一次",
  summary: "人类告知昨天晚上去了北京且玩得很开心，AI对此给予热情回应，提及北京的一些好玩之处，还进一步询问人类昨晚所去的具体地点及有无特别有意思的经历。随后人类回复去爬了长城，AI表示赞叹，称爬长城很不错，提及长城雄伟壮观能领略别样风景，并再次询问人类在爬长城过程中是否有遇到有意思的游客、看到特别美的景色等特别有趣的事儿。"
}
哇，那太棒啦！冬天的北京也别有一番风味呢。到时候可以去看看故宫的雪景，红墙黄瓦配上皑皑白雪，超级美，就像穿越回了古代的紫禁城。还有颐和园，冬天湖面结冰后，也是很有感觉哦。

那你这次冬天去北京，还是打算去爬长城呀，还是有其他想去的新地方呀？有没有特别期待在冬天的北京经历些什么有意思的事儿呢？

input {
  input: "我想去看升旗",
  summary: "人类告知昨天晚上去了北京且玩得很开心，AI热情回应并询问具体地点及有趣经历，人类回复去爬了长城，AI赞叹并进一步询问爬长城时的趣事。随后人类表示计划今年冬天再去一次北京，AI称冬天的北京别有风味，推荐可去看故宫雪景、颐和园结冰的湖面等，还询问人类此次冬天去北京是打算再爬长城还是去其他新地方，以及是否有特别期待在冬天的北京经历的有意